# Search grid generator

<a href="https://www.bellingcat.com"><img alt="Bellingcat logo: Discover Bellingcat" src="https://img.shields.io/badge/Discover%20Bellingcat-%20?style=for-the-badge&logo=data%3Aimage%2Fpng%3Bbase64%2CiVBORw0KGgoAAAANSUhEUgAAAA4AAAAYCAYAAADKx8xXAAABhGlDQ1BJQ0MgcHJvZmlsZQAAKJF9kT1Iw0AcxV9TS0UqDnZQEcxQneyiIo6likWwUNoKrTqYXPoFTRqSFBdHwbXg4Mdi1cHFWVcHV0EQ%2FABxdnBSdJES%2F5cUWsR4cNyPd%2Fced%2B8AoVllqtkTA1TNMtKJuJjLr4rBVwQwhhBEDEvM1JOZxSw8x9c9fHy9i%2FIs73N%2Fjn6lYDLAJxLHmG5YxBvEs5uWznmfOMzKkkJ8Tjxp0AWJH7kuu%2FzGueSwwDPDRjY9TxwmFktdLHcxKxsq8QxxRFE1yhdyLiuctzir1Tpr35O%2FMFTQVjJcpzmKBJaQRIo6klFHBVVYiNKqkWIiTftxD%2F%2BI40%2BRSyZXBYwcC6hBheT4wf%2Fgd7dmcXrKTQrFgcCLbX%2BMA8FdoNWw7e9j226dAP5n4Err%2BGtNYO6T9EZHixwBA9vAxXVHk%2FeAyx1g6EmXDMmR%2FDSFYhF4P6NvygODt0Dfmttbex%2BnD0CWulq%2BAQ4OgYkSZa97vLu3u7d%2Fz7T7%2BwHEU3LHAa%2FQ6gAAAAZiS0dEAAAAAAAA%2BUO7fwAAAAlwSFlzAAAuIwAALiMBeKU%2FdgAAAAd0SU1FB%2BgFHwwiMH4odB4AAAAZdEVYdENvbW1lbnQAQ3JlYXRlZCB3aXRoIEdJTVBXgQ4XAAAA50lEQVQ4y82SvWpCQRCFz25ERSJiCNqlUiS1b5AuEEiZIq1NOsGXCKms0wXSp9T6dskDiFikyiPc%2FrMZyf3FXSGQ0%2BzuzPl2ZoeVKgQ0gQ2wBVpVHlcDkjM5V%2FJ5nag6sJ%2FZX%2Bh%2FC7gEhqeAFKf7p1M9aB3b5oN1OomB7g1axUBPBr3GQHODHmOgqUF3MZAzKI2d4LWBV4H%2BMXDuJd1a7Cew1k7SwksaHC4LqNaw7aeX9GWHXkC1G1sTAS17Y3Kk2lnp4wNLiz0DrgLq8qt2MfmSSabAO%2FBBXp26dtrADPjOmN%2BAUdG7B3cE61l5hOZiAAAAAElFTkSuQmCC&logoColor=%23fff&color=%23000"></a><!--
--><a href="https://discord.gg/bellingcat"><img alt="Discord logo: Join our community" src="https://img.shields.io/badge/Join%20our%20community-%20?style=for-the-badge&logo=discord&logoColor=%23fff&color=%235865F2"></a><!--
--><a href="https://github.com/bellingcat/open-source-research-notebooks/blob/main/notebooks/bellingcat/search-grid-generator.ipynb"><img alt="Github logo: Explore the code" src="https://img.shields.io/badge/Explore%20the%20code-%20?style=for-the-badge&logo=github&color=%2329903b"></a>

A tool for generating a grid of rectangular polygons in a KML file. The resulting KML file can be opened in Google Earth and used for systematically searching an area.

# 0.0 Setup Grid (inputs)

In [1]:
# @title Create Grid { display-mode: "form" }

# @markdown ### ⬅️ Click to generate the grid once you've filled in the form

# @markdown Provide the coordinates of opposite corners of the area of interest:
lat_long_1 = "48.90529373066811, 2.2584719879516464"  # @param {type:"string"}
lat_long_2 = "48.81418021233939, 2.4172392962309246"  # @param {type:"string"}

# @markdown Set the approximate grid square size in km:
grid_size = 1  # @param {type:"number"}

# @markdown Customise the name of the output file:
file_name = "grid.kml"  # @param {type:"string"}

# 0.1 Imports

In [2]:
import numpy as np
from ipywidgets import widgets
from IPython import get_ipython
from IPython.display import display
from datetime import datetime

logfile = f"./grid_log_{datetime.now().strftime('%d%m%Y_%H%M%S')}.log"

# Install dependencies and enable widgets if running in colab
if "google.colab" in str(get_ipython()):
    !jupyter nbextension enable --py widgetsnbextension >> {logfile} 2>&1
    ![ ! -f "deps_installed" ] && pip install simplekml >> {logfile} 2>&1 && touch "deps_installed"

import simplekml

ModuleNotFoundError: No module named 'simplekml'

# 0.2 Grid generation Function

In [15]:
def generate_grid(
    lat1, lon1, lat2, lon2, lat_step=0.01, lon_step=0.01, file_name="grid.kml"
):
    # Create a kml object
    kml = simplekml.Kml()
    # create a folder
    fol = kml.newfolder(name="Grid")

    for lat in np.arange(lat1, lat2, lat_step):
        for lon in np.arange(lon1, lon2, lon_step):
            # Round the lat and lon values to 5 decimal places

            # Create a polygon
            pol = fol.newpolygon(name=f"({lat}, {lon})")
            pol.outerboundaryis = [
                (lon, lat),
                (lon + lon_step, lat),
                (lon + lon_step, lat + lat_step),
                (lon, lat + lat_step),
                (lon, lat),
            ]
            pol.style.linestyle.color = simplekml.Color.white
            pol.style.linestyle.width = 2
            # make the polygon transparent
            pol.style.polystyle.color = simplekml.Color.changealphaint(
                0, simplekml.Color.red
            )

    # Save the kml file
    kml.save(file_name)

# 1. The code
### 1.1 Filter, extract, and parse data

In [5]:
# Add kml extension to the file name if it doesn't exist
if not file_name.endswith(".kml"):
    file_name += ".kml"

# Extract the latitude and longitude
lat1, lon1 = lat_long_1.replace(" ", "").split(",")
lat2, lon2 = lat_long_2.replace(" ", "").split(",")

# Reorder the lat long values
lat1, lat2 = min(float(lat1), float(lat2)), max(float(lat1), float(lat2))
lon1, lon2 = min(float(lon1), float(lon2)), max(float(lon1), float(lon2))

# Convert grid_size to degrees for lat_step and lon_step separately
lon_step = grid_size / 111 / np.cos(np.radians((lat1 + lat2) / 2))
lat_step = grid_size / 111 

### 1.2 Generate the grid

In [1]:
generate_grid(
    lat1, lon1, lat2, lon2, lat_step=lat_step, lon_step=lon_step, file_name=file_name
)

NameError: name 'generate_grid' is not defined

### 1.3 Show download button if running in colab

In [12]:
if "google.colab" in str(get_ipython()):
    from google.colab import files
    btn = widgets.Button(description=f"Download {file_name}")
    btn.on_click(lambda x: files.download(file_name))
    display(btn)